In [1]:
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv("diabetes.csv")

In [3]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [11]:
data.shape

(768, 9)

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
X=data.drop(["Outcome"],axis=1)
Y=data[["Outcome"]]

In [8]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [9]:
X_train=StandardScaler().fit_transform(X_train)
X_test=StandardScaler().fit_transform(X_test)

In [10]:
X_train.shape

(614, 8)

In [12]:
n_nodes_hl1 = 100
n_nodes_hl2 = 50
#n_nodes_hl3 = 500
n_classes = 2
batch_size = 15

In [17]:
Y_train["Outcome"]=Y_train["Outcome"].astype("str")

In [18]:
Y_test["Outcome"]=Y_test["Outcome"].astype("str")

In [19]:
Y_train=pd.get_dummies(Y_train)
Y_test=pd.get_dummies(Y_test)

In [20]:
Y_test.head()

,Outcome_0,Outcome_1
661,0,1
122,1,0
113,1,0
14,0,1
529,1,0


In [28]:
Y_train=Y_train.values
Y_test=Y_test.values

In [23]:
x = tf.placeholder('float')
y = tf.placeholder('float')

In [33]:
n_nodes_hl1 = 100
n_nodes_hl2 = 50
#n_nodes_hl3 = 500

n_classes = 2
batch_size = 10

In [38]:
import numpy as np

In [26]:
hidden_1_layer = {'f_fum':n_nodes_hl1,
                  'weight':tf.Variable(tf.random_normal([len(X_train[0]), n_nodes_hl1])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl1]))}

hidden_2_layer = {'f_fum':n_nodes_hl2,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl2]))}

#hidden_3_layer = {'f_fum':n_nodes_hl3,
#                  'weight':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
#                  'bias':tf.Variable(tf.random_normal([n_nodes_hl3]))}

output_layer = {'f_fum':None,
                'weight':tf.Variable(tf.random_normal([n_nodes_hl2, n_classes])),
                'bias':tf.Variable(tf.random_normal([n_classes])),}


In [59]:
# Nothing changes
def neural_network_model(data):

    l1 = tf.add(tf.matmul(data,hidden_1_layer['weight']), hidden_1_layer['bias'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weight']), hidden_2_layer['bias'])
    l2 = tf.nn.relu(l2)

    #l3 = tf.add(tf.matmul(l2,hidden_3_layer['weight']), hidden_3_layer['bias'])
    #l3 = tf.nn.relu(l3)

    output = tf.matmul(l2,output_layer['weight']) + output_layer['bias']
    output=tf.nn.softmax(output)

    return output

In [61]:

def train_neural_network(x):
	prediction = neural_network_model(x)
	cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
	optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
	hm_epochs = 10
	with tf.Session() as sess:
		sess.run(tf.initialize_all_variables())
	    
		for epoch in range(hm_epochs):
			epoch_loss = 0
			i=0
			while i < len(X_test):
				start = i
				end = i+batch_size
				batch_x = np.array(X_train[start:end])
				batch_y = np.array(Y_train[start:end])

				_, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
				                                              y: batch_y})
				epoch_loss += c
				i+=batch_size
				
			print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)
		correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
		accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
		print(sess.run(prediction, feed_dict={x: X_test}))
		#print(pred)
		#prediction=tf.argmax(y,1)
		#print ("predictions", prediction.eval(feed_dict={x: X_test}, session=sess))
		print('Accuracy:',accuracy.eval({x:X_test, y:Y_test}))


In [62]:
train_neural_network(x)

Epoch 1 completed out of 10 loss: 14.6586507559
Epoch 2 completed out of 10 loss: 14.3468120098
Epoch 3 completed out of 10 loss: 14.1167266965
Epoch 4 completed out of 10 loss: 14.1118663549
Epoch 5 completed out of 10 loss: 14.1113693118
Epoch 6 completed out of 10 loss: 14.0959070325
Epoch 7 completed out of 10 loss: 14.128973484
Epoch 8 completed out of 10 loss: 13.9429790974
Epoch 9 completed out of 10 loss: 13.9689502716
Epoch 10 completed out of 10 loss: 13.8951455355
[[  1.00000000e+00   0.00000000e+00]
 [  3.90812884e-25   1.00000000e+00]
 [  2.05182131e-21   1.00000000e+00]
 [  4.42893508e-30   1.00000000e+00]
 [  8.29725356e-38   1.00000000e+00]
 [  2.89276311e-26   1.00000000e+00]
 [  0.00000000e+00   1.00000000e+00]
 [  0.00000000e+00   1.00000000e+00]
 [  0.00000000e+00   1.00000000e+00]
 [  1.00000000e+00   7.37044261e-24]
 [  0.00000000e+00   1.00000000e+00]
 [  0.00000000e+00   1.00000000e+00]
 [  7.93195427e-16   1.00000000e+00]
 [  4.51247420e-26   1.00000000e+00]
 [

In [63]:
#sess.run(pred, feed_dict={x: X_test})